- Signed Sage Convolution

In [1]:
import math
import json
import numpy as np
import pandas as pd
import time
import random
import argparse

from tqdm import trange
from scipy import sparse
from texttable import Texttable
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torch.nn.init as init

from torch.nn import Parameter
from torch_scatter import scatter_add, scatter_mean
from torch_geometric.utils import remove_self_loops, add_self_loops

### 构建模型

In [2]:
def uniform(size, tensor):
    stdv = 1.0 / math.sqrt(size)
    if tensor is not None:
        tensor.data.uniform_(-stdv, stdv)

In [3]:
class ListModule(torch.nn.Module):
    def __init__(self, *args):
        super(ListModule, self).__init__()
        idx = 0
        for module in args:
            self.add_module(str(idx), module)
            idx += 1

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self._modules):
            raise IndexError('index {} is out of range'.format(idx))
        it = iter(self._modules.values())
        for i in range(idx):
            next(it)
        return next(it)

    def __iter__(self):
        return iter(self._modules.values())

    def __len__(self):
        return len(self._modules)

In [4]:
class SignedSAGEConvolution(torch.nn.Module):
    def __init__(self, in_channels, out_channels, norm=True, norm_embed=True, bias=True):
        super(SignedSAGEConvolution, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.norm = norm
        self.norm_embed = norm_embed
        self.weight = Parameter(torch.Tensor(self.in_channels, out_channels))  # 32*3

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter("bias", None)

        self.reset_parameters()

    def reset_parameters(self):
        size = self.weight.size(0)
        uniform(size, self.weight)
        uniform(size, self.bias)

    def __repr__(self):
        return "{}({}, {})".format(self.__class__.__name__, self.in_channels, self.out_channels)

In [5]:
class SignedSAGEConvolutionBase(SignedSAGEConvolution):
    def forward(self, x, edge_index):
        edge_index, _ = remove_self_loops(edge_index, None)  # 去除自连接的边(A-A)
        row, col = edge_index  # 初始，终止节点编号;
        # scatter_mean: https://blog.csdn.net/StarfishCu/article/details/108853080
        if self.norm:
            out = scatter_mean(x[col], row, dim=0, dim_size=x.size(0))  # scatter_mean函数表示，row索引位置相加，返会结果; dim_size表示返回的节点，如果不在row中，则为0
        else:
            out = scatter_add(x[col], row, dim=0, dim_size=x.size(0))
        # 节点聚合周围邻居之后的信息; [5881, 64]
        out = torch.cat((out, x), 1)  # 将聚合特征和原始特征拼接
        out = torch.matmul(out, self.weight)

        if self.bias is not None:
            out = out + self.bias
        if self.norm_embed:
            out = F.normalize(out, p=2, dim=-1)
        return out

In [6]:
class SignedSAGEConvolutionDeep(SignedSAGEConvolution):
    def forward(self, x_1, x_2, edge_index_pos, edge_index_neg):
        edge_index_pos, _ = remove_self_loops(edge_index_pos, None)
        edge_index_pos, _ = add_self_loops(edge_index_pos, num_nodes=x_1.size(0))
        edge_index_neg, _ = remove_self_loops(edge_index_neg, None)
        edge_index_neg, _ = add_self_loops(edge_index_neg, num_nodes=x_2.size(0))

        row_pos, col_pos = edge_index_pos
        row_neg, col_neg = edge_index_neg

        if self.norm:  # pos: [x_1:balance features; x_2:unbalance features]  neg :[x_1:unbalance features; x_2:balance features]
            out_1 = scatter_mean(x_1[col_pos], row_pos, dim=0, dim_size=x_1.size(0))
            out_2 = scatter_mean(x_2[col_neg], row_neg, dim=0, dim_size=x_2.size(0))
        else:
            out_1 = scatter_add(x_1[col_pos], row_pos, dim=0, dim_size=x_1.size(0))
            out_2 = scatter_add(x_2[col_neg], row_neg, dim=0, dim_size=x_2.size(0))

        out = torch.cat((out_1, out_2, x_1), 1)
        out = torch.matmul(out, self.weight)
        if self.bias is not None:
            out = out + self.bias

        if self.norm_embed:
            out = F.normalize(out, p=2, dim=-1)
        return out

In [7]:
def calculate_auc(targets, predictions, edges):
    neg_ratio = len(edges["negative_edges"]) / edges["ecount"]
    targets = [0 if target == 1 else 1 for target in targets]
    auc = roc_auc_score(targets, predictions)
    f1 = f1_score(targets, [1 if p > neg_ratio else 0 for p in  predictions])
    return auc, f1

In [8]:
def create_general_features(args):
    X = np.array(pd.read_csv(args.features_path))
    return X

def create_spectral_features(args, positive_edges, negative_edges, node_count):
    p_edges = positive_edges + [[edge[1], edge[0]] for edge in positive_edges]  # 转换成无向图的点边关系
    n_edges = negative_edges + [[edge[1], edge[0]] for edge in negative_edges]
    train_edges = p_edges + n_edges  # 所有点边信息
    index_1 = [edge[0] for edge in train_edges]  # src
    index_2 = [edge[1] for edge in train_edges]  # dst
    values = [1] * len(p_edges) + [-1] * len(n_edges)  # label定义
    shaping = (node_count, node_count)  # 节点数量，构造点边关系矩阵
    signed_A = sparse.csr_matrix(sparse.coo_matrix((values, (index_1, index_2)),  # 构建稀疏邻接矩阵
                                                   shape=shaping,
                                                   dtype=np.float32))
    # SVD分解，构造节点特征就是节点邻接矩阵的降维
    svd = TruncatedSVD(
        n_components=args.reduction_dimensions,
        n_iter=args.reduction_iterations,
        random_state=args.seed
    )
    svd.fit(signed_A)
    X = svd.components_.T  # 降维后的特征
    return X

In [9]:
def setup_features(args, positive_edges, negative_edges, node_count):
    if args.spectral_features:
        X = create_spectral_features(args, positive_edges, negative_edges, node_count)  # 构造节点特征，为邻接矩阵降维结果
    else:
        X = create_general_features(args)  # origin
    return X

In [10]:
class SignedGraphConvolutionalNetwork(torch.nn.Module):
    """
    Signed Graph Convolutional Network Class.
    For details see: Signed Graph Convolutional Network.
    Tyler Derr, Yao Ma, and Jiliang Tang ICDM, 2018.
    https://arxiv.org/abs/1808.06354
    """
    def __init__(self, device, args, X):
        super(SignedGraphConvolutionalNetwork, self).__init__()
        """
        SGCN Initialization.
        :param device: Device for calculations.
        :param args: Arguments object.
        :param X: Node features.
        """
        self.args = args
        torch.manual_seed(self.args.seed)
        self.device = device
        self.X = X
        self.setup_layers()  # 构建SGCN，Layer

    def setup_layers(self):
        """
        Adding Base Layers, Deep Signed GraphSAGE layers.
        Assing Regression Parameters if the model is not a single layer model.
        """
        self.nodes = range(self.X.shape[0])
        self.neurons = self.args.layers  # 网络层参数
        self.layers = len(self.neurons)  # 网络层数
        self.positive_base_aggregator = SignedSAGEConvolutionBase(self.X.shape[1]*2,  # 64*2 -> 32  # 构建第一层SGCN
                                                                  self.neurons[0]).to(self.device)

        self.negative_base_aggregator = SignedSAGEConvolutionBase(self.X.shape[1]*2,
                                                                  self.neurons[0]).to(self.device)
        self.positive_aggregators = []
        self.negative_aggregators = []
        for i in range(1, self.layers):  # l>1 层SGCN
            self.positive_aggregators.append(SignedSAGEConvolutionDeep(3*self.neurons[i-1],
                                                                       self.neurons[i]).to(self.device))

            self.negative_aggregators.append(SignedSAGEConvolutionDeep(3*self.neurons[i-1],
                                                                       self.neurons[i]).to(self.device))

        self.positive_aggregators = ListModule(*self.positive_aggregators)  # 给予网络层编号
        self.negative_aggregators = ListModule(*self.negative_aggregators)
        self.regression_weights = Parameter(torch.Tensor(4*self.neurons[-1], 3))  # 4*32=128; edge_label参数
        init.xavier_normal_(self.regression_weights)

    def calculate_regression_loss(self, z, target):
        """
        Calculating the regression loss for all pairs of nodes.
        :param z: Hidden vertex representations.
        :param target: Target vector.
        :return loss_term: Regression loss.
        :return predictions_soft: Predictions for each vertex pair.
        """
        pos = torch.cat((self.positive_z_i, self.positive_z_j), 1)  # 拼接pos: [i,j]
        neg = torch.cat((self.negative_z_i, self.negative_z_j), 1)  # 拼接neg: [i,j]

        surr_neg_i = torch.cat((self.negative_z_i, self.negative_z_k), 1)  # 不连接边: [i,k]
        surr_neg_j = torch.cat((self.negative_z_j, self.negative_z_k), 1)
        surr_pos_i = torch.cat((self.positive_z_i, self.positive_z_k), 1)
        surr_pos_j = torch.cat((self.positive_z_j, self.positive_z_k), 1)

        features = torch.cat((pos, neg, surr_neg_i, surr_neg_j, surr_pos_i, surr_pos_j))
        predictions = torch.mm(features, self.regression_weights)
        predictions_soft = F.log_softmax(predictions, dim=1)
        loss_term = F.nll_loss(predictions_soft, target)  # 对数损失
        return loss_term, predictions_soft

    def calculate_positive_embedding_loss(self, z, positive_edges):
        """
        Calculating the loss on the positive edge embedding distances
        :param z: Hidden vertex representation.
        :param positive_edges: Positive training edges.
        :return loss_term: Loss value on positive edge embedding.
        """
        self.positive_surrogates = [random.choice(self.nodes) for node in range(positive_edges.shape[1])]  # 随机positive数量相同的选择不连接节点
        self.positive_surrogates = torch.from_numpy(np.array(self.positive_surrogates, dtype=np.int64).T)
        self.positive_surrogates = self.positive_surrogates.type(torch.long).to(self.device)  # 转换成torch格式
        positive_edges = torch.t(positive_edges)  # positive_edge转置
        self.positive_z_i = z[positive_edges[:, 0], :]  # row的embedding
        self.positive_z_j = z[positive_edges[:, 1], :]  # col的embedding
        self.positive_z_k = z[self.positive_surrogates, :]  # 随机的节点的embedding
        norm_i_j = torch.norm(self.positive_z_i-self.positive_z_j, 2, 1, True).pow(2)
        norm_i_k = torch.norm(self.positive_z_i-self.positive_z_k, 2, 1, True).pow(2)
        term = norm_i_j-norm_i_k
        term[term < 0] = 0
        loss_term = term.mean()
        return loss_term

    def calculate_negative_embedding_loss(self, z, negative_edges):
        """
        Calculating the loss on the negative edge embedding distances
        :param z: Hidden vertex representation.
        :param negative_edges: Negative training edges.
        :return loss_term: Loss value on negative edge embedding.
        """
        self.negative_surrogates = [random.choice(self.nodes) for node in range(negative_edges.shape[1])]
        self.negative_surrogates = torch.from_numpy(np.array(self.negative_surrogates, dtype=np.int64).T)
        self.negative_surrogates = self.negative_surrogates.type(torch.long).to(self.device)
        negative_edges = torch.t(negative_edges)
        self.negative_z_i = z[negative_edges[:, 0], :]
        self.negative_z_j = z[negative_edges[:, 1], :]
        self.negative_z_k = z[self.negative_surrogates, :]
        norm_i_j = torch.norm(self.negative_z_i-self.negative_z_j, 2, 1, True).pow(2)
        norm_i_k = torch.norm(self.negative_z_i-self.negative_z_k, 2, 1, True).pow(2)
        term = norm_i_k-norm_i_j
        term[term < 0] = 0
        loss_term = term.mean()
        return loss_term

    def calculate_loss_function(self, z, positive_edges, negative_edges, target):
        """
        Calculating the embedding losses, regression loss and weight regularization loss.
        :param z: Node embedding.
        :param positive_edges: Positive edge pairs.
        :param negative_edges: Negative edge pairs.
        :param target: Target vector.
        :return loss: Value of loss.
        """
        loss_term_1 = self.calculate_positive_embedding_loss(z, positive_edges)
        loss_term_2 = self.calculate_negative_embedding_loss(z, negative_edges)
        regression_loss, self.predictions = self.calculate_regression_loss(z, target)
        loss_term = regression_loss+self.args.lamb*(loss_term_1+loss_term_2)
        return loss_term

    def forward(self, positive_edges, negative_edges, target):
        """
        Model forward propagation pass. Can fit deep and single layer SGCN models.
        :param positive_edges: Positive edges.
        :param negative_edges: Negative edges.
        :param target: Target vectors.
        :return loss: Loss value.
        :return self.z: Hidden vertex representations.
        """
        self.h_pos, self.h_neg = [], []
        self.h_pos.append(torch.tanh(self.positive_base_aggregator(self.X, positive_edges)))  # self.X=[5881,64]; positive_edges=[2,14624]
        self.h_neg.append(torch.tanh(self.negative_base_aggregator(self.X, negative_edges)))
        for i in range(1, self.layers):
            self.h_pos.append(torch.tanh(self.positive_aggregators[i-1](self.h_pos[i-1], self.h_neg[i-1], positive_edges, negative_edges)))
            self.h_neg.append(torch.tanh(self.negative_aggregators[i-1](self.h_neg[i-1], self.h_pos[i-1], positive_edges, negative_edges)))
        self.z = torch.cat((self.h_pos[-1], self.h_neg[-1]), 1)
        loss = self.calculate_loss_function(self.z, positive_edges, negative_edges, target)
        return loss, self.z

In [11]:
class SignedGCNTrainer(object):
    """
    Object to train and score the SGCN, log the model behaviour and save the output.
    """
    def __init__(self, args, edges):
        """
        Constructing the trainer instance and setting up logs.
        :param args: Arguments object.
        :param edges: Edge data structure with positive and negative edges separated.
        """
        self.args = args
        self.edges = edges
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.setup_logs()

    def setup_logs(self):
        """
        Creating a log dictionary.
        """
        self.logs = {}
        self.logs["parameters"] = vars(self.args)
        self.logs["performance"] = [["Epoch", "AUC", "F1"]]
        self.logs["training_time"] = [["Epoch", "Seconds"]]

    def setup_dataset(self):
        """
        Creating train and test split.
        """
        self.positive_edges, self.test_positive_edges = train_test_split(self.edges["positive_edges"],  # postive的点边关系
                                                                         test_size=self.args.test_size)

        self.negative_edges, self.test_negative_edges = train_test_split(self.edges["negative_edges"],
                                                                         test_size=self.args.test_size)
        self.ecount = len(self.positive_edges + self.negative_edges)  # 训练样本数量
        # SVD分解，邻接矩阵，为节点特征;
        self.X = setup_features(self.args,
                                self.positive_edges,  # 训练样本
                                self.negative_edges,
                                self.edges["ncount"])  # 点的数量
        # positive样本的点边关系转换成torch类型
        self.positive_edges = torch.from_numpy(np.array(self.positive_edges,
                                                        dtype=np.int64).T).type(torch.long).to(self.device)
        # negative样本的点边关系
        self.negative_edges = torch.from_numpy(np.array(self.negative_edges,
                                                        dtype=np.int64).T).type(torch.long).to(self.device)
        # label: [0],[1] 两个部分; [2]是两倍的点边数量-表示不连接的边; !!  np.array([0]*len(self.positive_edges) + [1]*len(self.negative_edges) + [2]*(self.ecount*2))
        self.y = np.array([0]*len(self.positive_edges[0]) + [1]*len(self.negative_edges[0]) + [2]*(self.ecount*2))  #np.array([0 if i < int(self.ecount/2) else 1 for i in range(self.ecount)]+[2]*(self.ecount*2))
        self.y = torch.from_numpy(self.y).type(torch.LongTensor).to(self.device)
        self.X = torch.from_numpy(self.X).float().to(self.device)  # [5881,64]的input features

    def score_model(self, epoch):
        """
        Score the model on the test set edges in each epoch.
        :param epoch: Epoch number.
        """
        loss, self.train_z = self.model(self.positive_edges, self.negative_edges, self.y)
        score_positive_edges = torch.from_numpy(np.array(self.test_positive_edges, dtype=np.int64).T).type(torch.long).to(self.device)
        score_negative_edges = torch.from_numpy(np.array(self.test_negative_edges, dtype=np.int64).T).type(torch.long).to(self.device)
        test_positive_z = torch.cat((self.train_z[score_positive_edges[0, :], :], self.train_z[score_positive_edges[1, :], :]), 1)
        test_negative_z = torch.cat((self.train_z[score_negative_edges[0, :], :], self.train_z[score_negative_edges[1, :], :]), 1)
        scores = torch.mm(torch.cat((test_positive_z, test_negative_z), 0), self.model.regression_weights.to(self.device))
        probability_scores = torch.exp(F.softmax(scores, dim=1))
        predictions = probability_scores[:, 0]/probability_scores[:, 0:2].sum(1)
        predictions = predictions.cpu().detach().numpy()
        targets = [0]*len(self.test_positive_edges) + [1]*len(self.test_negative_edges)
        auc, f1 = calculate_auc(targets, predictions, self.edges)
        self.logs["performance"].append([epoch+1, auc, f1])

    def create_and_train_model(self):
        """
        Model training and scoring.
        """
        print("\nTraining started.\n")
        self.model = SignedGraphConvolutionalNetwork(self.device, self.args, self.X).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),
                                          lr=self.args.learning_rate,
                                          weight_decay=self.args.weight_decay)
        self.model.train()
        self.epochs = trange(self.args.epochs, desc="Loss")
        for epoch in self.epochs:
            start_time = time.time()
            self.optimizer.zero_grad()
            loss, _ = self.model(self.positive_edges, self.negative_edges, self.y)
            loss.backward()
            self.epochs.set_description("SGCN (Loss=%g)" % round(loss.item(), 4))
            self.optimizer.step()
            self.logs["training_time"].append([epoch+1, time.time()-start_time])
            if self.args.test_size > 0:
                self.score_model(epoch)

    def save_model(self):
        """
        Saving the embedding and model weights.
        """
        print("\nEmbedding is saved.\n")
        self.train_z = self.train_z.cpu().detach().numpy()
        embedding_header = ["id"] + ["x_" + str(x) for x in range(self.train_z.shape[1])]
        self.train_z = np.concatenate([np.array(range(self.train_z.shape[0])).reshape(-1, 1), self.train_z], axis=1)
        self.train_z = pd.DataFrame(self.train_z, columns=embedding_header)
        self.train_z.to_csv(self.args.embedding_path, index=None)
        
        print("\nRegression weights are saved.\n")
        self.regression_weights = self.model.regression_weights.cpu().detach().numpy().T
        regression_header = ["x_" + str(x) for x in range(self.regression_weights.shape[1])]
        self.regression_weights = pd.DataFrame(self.regression_weights, columns=regression_header)
        self.regression_weights.to_csv(self.args.regression_weights_path, index=None)

In [12]:
def read_graph(args):
    dataset = pd.read_csv(args.edge_path).values.tolist()
    edges = {}
    edges["positive_edges"] = [edge[0:2] for edge in dataset if edge[2] == 1]  # postitive的edge
    edges["negative_edges"] = [edge[0:2] for edge in dataset if edge[2] == -1]  # negative的edge
    edges["ecount"] = len(dataset)  # 数据数量
    edges["ncount"] = len(set([edge[0] for edge in dataset]+[edge[1] for edge in dataset]))  # 所有节点的数量
    return edges

In [13]:
def tab_printer(args):
    args = vars(args)
    keys = sorted(args.keys())
    t = Texttable()
    t.add_rows([["Parameter", "Value"]])
    t.add_rows([[k.replace("_", " ").capitalize(), args[k]] for k in keys])
    print(t.draw())

In [14]:
def tab_printer(args):
    args = vars(args)
    keys = sorted(args.keys())
    t = Texttable()
    t.add_rows([["Parameter", "Value"]])
    t.add_rows([[k.replace("_", " ").capitalize(), args[k]] for k in keys])
    print(t.draw())

In [15]:
def score_printer(logs):
    t = Texttable()
    t.add_rows([per for i, per in enumerate(logs["performance"]) if i % 10 == 0])
    print(t.draw())

In [16]:
def save_logs(args, logs):
    with open(args.log_path, "w") as f:
        json.dump(logs, f)

### 参数设置

In [17]:
def parameter_parser():
    parser = argparse.ArgumentParser(description="Run SGCN.")
    
    parser.add_argument("--edge-path", nargs="?", default=r"C:\Users\sss\Desktop\SGCN/input/bitcoin_otc.csv", help="Edge list csv.")
    parser.add_argument("--features-path", nargs="?", default=r"C:\Users\sss\Desktop\SGCN/input/bitcoin_otc.csv", help="Edge list csv.")
    parser.add_argument("--embedding-path", nargs="?", default=r"C:\Users\sss\Desktop\SGCN/output/embedding/bitcoin_otc_sgcn.csv", help="Target embedding csv.")
    parser.add_argument("--regression-weights-path", nargs="?", default=r"C:\Users\sss\Desktop\SGCN/output/weights/bitcoin_otc_sgcn.csv", help="Regression weights csv.")
    # parser.add_argument("--log-path", nargs="?", default=r"C:\Users\sss\Desktop\SGCN/logs/bitcoin_otc_logs.json", help="Log json.")
    parser.add_argument("--epochs", type=int, default=100, help="Number of training epochs. Default is 100.")
    parser.add_argument("--reduction-iterations", type=int, default=30, help="Number of SVD iterations. Default is 30.")
    parser.add_argument("--reduction-dimensions", type=int, default=64, help="Number of SVD feature extraction dimensions. Default is 64.")
    parser.add_argument("--seed", type=int, default=42, help="Random seed for sklearn pre-training. Default is 42.")
    parser.add_argument("--lamb", type=float, default=1.0, help="Embedding regularization parameter. Default is 1.0.")
    parser.add_argument("--test-size", type=float, default=0.2, help="Test dataset size. Default is 0.2.")
    parser.add_argument("--learning-rate", type=float, default=0.01, help="Learning rate. Default is 0.01.")
    parser.add_argument("--weight-decay", type=float, default=10**-5, help="Learning rate. Default is 10^-5.")
    parser.add_argument("--layers", nargs="+", type=int, help="Layer dimensions separated by space. E.g. 32 32.")
    parser.add_argument("--spectral-features", dest="spectral_features", action="store_true")
    parser.add_argument("--general-features", dest="spectral_features", action="store_false")
    
    parser.set_defaults(spectral_features=True)
    parser.set_defaults(layers=[32, 32])

    return parser.parse_args(args=[])

In [18]:
args = parameter_parser()

In [19]:
tab_printer(args)

+-------------------------+----------------------------------------------------+
|        Edge path        |  C:\Users\sss\Desktop\SGCN/input/bitcoin_otc.csv   |
+=========================+====================================================+
| Embedding path          | C:\Users\sss\Desktop\SGCN/output/embedding/bitcoin |
|                         | _otc_sgcn.csv                                      |
+-------------------------+----------------------------------------------------+
| Epochs                  | 100                                                |
+-------------------------+----------------------------------------------------+
| Features path           | C:\Users\sss\Desktop\SGCN/input/bitcoin_otc.csv    |
+-------------------------+----------------------------------------------------+
| Lamb                    | 1                                                  |
+-------------------------+----------------------------------------------------+
| Layers                  | 

#### 重置参数

In [20]:
# 输入数据
args.edge_path = r'C:\Users\sss\Desktop\SGCN/input/bitcoin_otc.csv'
args.features_path = r'C:\Users\sss\Desktop\SGCN/input/bitcoin_otc.csv'

# 文件保存
args.embedding_path = r'C:\Users\sss\Desktop\SGCN/output/embedding/bitcoin_otc_sgcn.csv'
args.regression_weights_path = r'C:\Users\sss\Desktop\SGCN/output/weights/bitcoin_otc_sgcn.csv'
# args.epochs = 1  # 测试模型【模型成功即可修改】

In [21]:
edges = read_graph(args)  # 导入训练数据

In [22]:
trainer = SignedGCNTrainer(args, edges)
trainer.setup_dataset()
trainer.create_and_train_model()

Loss:   0%|                                                                                    | 0/100 [00:00<?, ?it/s]


Training started.



SGCN (Loss=0.6193): 100%|████████████████████████████████████████████████████████████| 100/100 [00:54<00:00,  1.85it/s]


In [23]:
if args.test_size > 0:
    trainer.save_model()
    score_printer(trainer.logs)
    # save_logs(args, trainer.logs)


Embedding is saved.


Regression weights are saved.

+-------+-------+-------+
| Epoch |  AUC  |  F1   |
+=======+=======+=======+
| 10    | 0.708 | 0.921 |
+-------+-------+-------+
| 20    | 0.695 | 0.921 |
+-------+-------+-------+
| 30    | 0.662 | 0.921 |
+-------+-------+-------+
| 40    | 0.700 | 0.921 |
+-------+-------+-------+
| 50    | 0.757 | 0.921 |
+-------+-------+-------+
| 60    | 0.774 | 0.921 |
+-------+-------+-------+
| 70    | 0.776 | 0.921 |
+-------+-------+-------+
| 80    | 0.788 | 0.921 |
+-------+-------+-------+
| 90    | 0.786 | 0.921 |
+-------+-------+-------+
| 100   | 0.797 | 0.921 |
+-------+-------+-------+
